# Preparing the Dataset

In [ ]:
import zipfile
import os
import torch
import numpy as np
from sklearn.preprocessing import MinMaxScaler


gesture_zip_path = 'home/Downloads/Gesture.zip' 
gesture_extract_path = 'Gesture_data/'

# Step 1: Unzip Gesture.zip
with zipfile.ZipFile(gesture_zip_path, 'r') as zip_ref:
    zip_ref.extractall(gesture_extract_path)

# Load gesture data assuming files are in PyTorch tensor format
gesture_data = {
    'train': torch.load(os.path.join(gesture_extract_path, 'train.pt')),
    'val': torch.load(os.path.join(gesture_extract_path, 'val.pt')),
    'test': torch.load(os.path.join(gesture_extract_path, 'test.pt'))
}

# Preprocess function for handling missing values and scaling
def preprocess_data(data_dict):
    preprocessed_data = {}
    scaler = MinMaxScaler()

    for split, data in data_dict.items():
        data_np = data.numpy()
        
        # Impute missing values with column mean
        if np.isnan(data_np).any():
            col_means = np.nanmean(data_np, axis=0)
            inds = np.where(np.isnan(data_np))
            data_np[inds] = np.take(col_means, inds[1])
        
        # Scale data
        data_np = scaler.fit_transform(data_np)
        preprocessed_data[split] = torch.tensor(data_np, dtype=torch.float32)
    
    return preprocessed_data

# Apply preprocessing
gesture_preprocessed_data = preprocess_data(gesture_data)


# Implementation of Self-Supervised Learning

In [ ]:
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim

# Load HAR data for self-supervised learning pre-training
har_zip_path = 'path_to_your_HAR.zip'
har_extract_path = 'HAR_data/'
with zipfile.ZipFile(har_zip_path, 'r') as zip_ref:
    zip_ref.extractall(har_extract_path)

har_data = {
    'train': torch.load(os.path.join(har_extract_path, 'train.pt')),
    'val': torch.load(os.path.join(har_extract_path, 'val.pt')),
    'test': torch.load(os.path.join(har_extract_path, 'test.pt'))
}

# Self-supervised model using a simple contrastive learning architecture
class ContrastiveModel(nn.Module):
    def __init__(self, input_dim, feature_dim=128):
        super(ContrastiveModel, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, feature_dim)
        )
    
    def forward(self, x):
        return self.encoder(x)

# Define contrastive loss
def contrastive_loss(features, temperature=0.5):
    cos_sim = nn.CosineSimilarity(dim=-1)
    labels = torch.arange(len(features)).long()
    logits = cos_sim(features.unsqueeze(1), features.unsqueeze(0)) / temperature
    return nn.CrossEntropyLoss()(logits, labels)

# Training loop for self-supervised learning
def pretrain_contrastive_model(har_data, input_dim):
    model = ContrastiveModel(input_dim)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    
    train_loader = DataLoader(har_data['train'], batch_size=32, shuffle=True)
    
    model.train()
    for epoch in range(20):
        total_loss = 0
        for data in train_loader:
            optimizer.zero_grad()
            features = model(data)
            loss = contrastive_loss(features)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_loader)}")
    
    return model

# Pretrain the model
input_dim = har_data['train'].shape[1]  # Assuming 2D input (samples x features)
contrastive_model = pretrain_contrastive_model(har_data, input_dim)


# Building a Classification Model

In [ ]:
class ClassifierModel(nn.Module):
    def __init__(self, feature_dim, num_classes):
        super(ClassifierModel, self).__init__()
        self.encoder = contrastive_model.encoder  # Reuse pre-trained encoder
        self.classifier = nn.Linear(feature_dim, num_classes)

    def forward(self, x):
        features = self.encoder(x)
        return self.classifier(features)

# Define the training function for supervised learning
def train_classifier(model, data, num_epochs=10):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    
    train_loader = DataLoader(data['train'], batch_size=32, shuffle=True)
    
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_loader)}")

# Instantiate and train classifier
num_classes = 8  # Adjust based on UWaveGestureLibrary class count
classifier_model = ClassifierModel(feature_dim=128, num_classes=num_classes)
train_classifier(classifier_model, gesture_preprocessed_data)


# Performance Verification

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def evaluate_model(model, data):
    model.eval()
    with torch.no_grad():
        predictions, labels = [], []
        for inputs, targets in DataLoader(data['test'], batch_size=32):
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            predictions.extend(preds.numpy())
            labels.extend(targets.numpy())
    
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')
    print(f"Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}")

# Evaluate the classifier
evaluate_model(classifier_model, gesture_preprocessed_data)
